### Dependencies 

In [15]:
## Fix for Jupyter Notebook only — do NOT use in production
## Allows nested async loops (e.g., for asyncio in notebooks) — not needed in normal .py files
## For production please use normal .py files!!! 
## See: https://github.com/erdewit/nest_asyncio for more details
import nest_asyncio
nest_asyncio.apply()

In [16]:
from starfish import StructuredLLM, data_factory
from starfish.common.env_loader import load_env_file

load_env_file()

2025-04-16 22:24:34 | INFO     | starfish.common.env_loader | env_loader.py:52 | Loaded 9 environment variables from /Users/zhengisamazing/1.python_dir/starfish/.env


True

### Structured LLM - Single
#### 1. Model provider LLM Call

In [17]:
first_llm = StructuredLLM(
    model_name="openai/gpt-4o-mini",
    prompt="Facts about city {{city_name}}.",
    output_schema=[{"name": "question", "type": "str"}, {"name": "answer", "type": "str"}],
    model_kwargs={"temperature": 0.7},
)

first_response = await first_llm.run(city_name="New York")
first_response.data

[{'question': 'What is the population of New York City?',
  'answer': 'As of 2023, New York City has an estimated population of over 8.4 million people, making it the most populous city in the United States.'}]

In [18]:
print(first_llm.render_prompt_printable(city_name="New York", num_records=5))


📝 CONSTRUCTED MESSAGES:

Role: user
Content:
Facts about city New York.


You are asked to generate exactly 5 records and please return the data in the following JSON format:
[
    {
    "question": ""  //  (required),
    "answer": ""  //  (required)
    }
    ...
]

Required fields: question, answer


End of prompt



#### 2. Customized Openai Compatible Model provider LLM Call

In [19]:
first_llm = StructuredLLM(
    model_name="hyperbolic/deepseek-ai/DeepSeek-V3-0324",
    prompt="Facts about city {{city_name}}.",
    output_schema=[{"name": "question", "type": "str"}, {"name": "answer", "type": "str"}],
    model_kwargs={"temperature": 0.7},
)

first_response = await first_llm.run(city_name="New York", num_records=5)
first_response.data

[{'question': 'What is the nickname of New York City?',
  'answer': 'The Big Apple'},
 {'question': 'Which famous park is located in the center of Manhattan?',
  'answer': 'Central Park'},
 {'question': 'What is the name of the tallest building in New York City?',
  'answer': 'One World Trade Center'},
 {'question': 'Which iconic statue is located in New York Harbor?',
  'answer': 'Statue of Liberty'},
 {'question': 'What is the name of the famous theater district in Manhattan?',
  'answer': 'Broadway'}]

#### 3. Local LLM

In [20]:
### Local model
first_llm = StructuredLLM(
    model_name="ollama/gemma3:1b",
    prompt="Facts about city {{city_name}}.",
    output_schema=[{"name": "question", "type": "str"}, {"name": "answer", "type": "str"}],
    model_kwargs={"temperature": 0.7},
)

first_response = await first_llm.run(city_name="New York", num_records=5)
first_response.data

2025-04-16 22:24:46 | INFO     | starfish.llm.proxy.litellm_adapter | litellm_adapter.py:94 | Ensuring Ollama model gemma3:1b is ready...
2025-04-16 22:24:46 | INFO     | starfish.llm.backend.ollama_adapter | ollama_adapter.py:63 | Starting Ollama server...
2025-04-16 22:24:46 | INFO     | starfish.llm.backend.ollama_adapter | ollama_adapter.py:79 | Ollama server started successfully
2025-04-16 22:24:46 | INFO     | starfish.llm.backend.ollama_adapter | ollama_adapter.py:129 | Found model gemma3:1b
2025-04-16 22:24:46 | INFO     | starfish.llm.backend.ollama_adapter | ollama_adapter.py:232 | Model gemma3:1b is already available
2025-04-16 22:24:46 | INFO     | starfish.llm.proxy.litellm_adapter | litellm_adapter.py:103 | Model gemma3:1b is ready, making API call...


[{'question': 'What is the population of New York City?',
  'answer': 'As of 2023, the population of New York City is approximately 8.8 million people.'}]

In [21]:
### Clean it up
from starfish.llm.backend.ollama_adapter import stop_ollama_server

await stop_ollama_server()

2025-04-16 22:24:51 | INFO     | starfish.llm.backend.ollama_adapter | ollama_adapter.py:254 | Stopping Ollama server...
2025-04-16 22:24:52 | INFO     | starfish.llm.backend.ollama_adapter | ollama_adapter.py:305 | Ollama server stopped successfully


True

### Structured LLM - Workflow
#### 1. Two LLM

In [22]:
from starfish import StructuredLLM
from starfish.llm.utils import merge_structured_outputs

first_llm = StructuredLLM(
    model_name="openai/gpt-4o-mini",
    prompt="Facts about city {{city_name}}.",
    output_schema=[{"name": "question", "type": "str"}, {"name": "answer", "type": "str"}],
)

first_response = await first_llm.run(city_name="New York", num_records=5)


second_llm = StructuredLLM(
    model_name="openai/gpt-4o-mini",
    prompt="""You will be given a list of question and answer pairs,
please rate each individually about its accuracy, funny and conciseness.
rating are from 1 to 10, 1 being the worst and 10 being the best.
lets also rank them among themself so from 1 being the best.
Here is question and answer pairs: {{QnA_pairs}}""",
    output_schema=[
        {"name": "accuracy", "type": "int"},
        {"name": "funny", "type": "int"},
        {"name": "conciseness", "type": "int"},
        {"name": "rank", "type": "int"},
    ],
    model_kwargs={"temperature": 1},
)

second_response = await second_llm.run(QnA_pairs=first_response.data)

### Merge result:
merge_structured_outputs(first_response.data, second_response.data)

[{'question': 'What is the population of New York City as of 2023?',
  'answer': 'As of 2023, the estimated population of New York City is approximately 8.5 million people.',
  'accuracy': 9,
  'funny': 2,
  'conciseness': 9,
  'rank': 1},
 {'question': 'What is the tallest building in New York City?',
  'answer': 'The tallest building in New York City is One World Trade Center, which stands at 1,776 feet tall.',
  'accuracy': 9,
  'funny': 2,
  'conciseness': 9,
  'rank': 2},
 {'question': 'What famous park is located in the heart of Manhattan?',
  'answer': 'Central Park is the famous park located in the heart of Manhattan.',
  'accuracy': 10,
  'funny': 2,
  'conciseness': 10,
  'rank': 3},
 {'question': 'Which bridge connects Manhattan and Brooklyn?',
  'answer': 'The Brooklyn Bridge connects Manhattan and Brooklyn.',
  'accuracy': 10,
  'funny': 2,
  'conciseness': 10,
  'rank': 4},
 {'question': 'What is the nickname of New York City?',
  'answer': "New York City is commonly know